In [1]:
import xarray as xr
import plotly.express as px
import plotly.graph_objects as go
import pandas as pd
import cf_xarray.units  # noqa: F401
import pint_xarray  # noqa: F401
from datetime import datetime

pd.options.plotting.backend = "plotly"

---

## Load datasets


In [2]:
data_bats = xr.open_dataset(
    "../stations/bats/3_post_processed/bats_product.zarr", engine="zarr"
)
data_bats

<xarray.Dataset> Size: 16MB
Dimensions:                        (time: 4428, depth: 8, latitude: 1,
                                    longitude: 1, is_day: 2, sieve_size: 5)
Coordinates:
  * depth                          (depth) float64 64B 50.0 100.0 ... 400.0
  * is_day                         (is_day) bool 2B False True
  * latitude                       (latitude) float64 8B 31.6
  * longitude                      (longitude) float64 8B -64.2
  * sieve_size                     (sieve_size) float64 40B 0.2 0.5 1.0 2.0 5.0
  * time                           (time) datetime64[ns] 35kB 1988-10-21T04:5...
Data variables: (12/37)
    CO2                            (time, depth, latitude, longitude) float64 283kB ...
    CTD_salinity                   (time, depth, latitude, longitude) float64 283kB ...
    POC                            (time, depth, latitude, longitude) float64 283kB ...
    PON                            (time, depth, latitude, longitude) float64 283kB ...
    POP                            (time, depth, latitude, longitude) float64 283kB ...
    TN                             (time, depth, latitude, longitude) float64 283kB ...
    ...                             ...
    silicate                       (time, depth, latitude, longitude) float64 283kB ...
    synechococcus                  (time, depth, latitude, longitude) float64 283kB ...
    t0                             (time, depth, latitude, longitude) float64 283kB ...
    temp                           (time, depth, latitude, longitude) float64 283kB ...
    total_dissolved_phosphorus     (time, depth, latitude, longitude) float64 283kB ...
    wet_weight_vol_water_ratio     (time, is_day, depth, latitude, longitude, sieve_size) float64 3MB ...

In [3]:
data_hot = xr.open_dataset(
    "../stations/hot/3_post_processed/hot_product.zarr", engine="zarr"
)
data_hot

<xarray.Dataset> Size: 4MB
Dimensions:     (time: 687, is_day: 2, latitude: 1, longitude: 1, depth: 8,
                 sieve_size: 5)
Coordinates:
  * depth       (depth) int64 64B 50 100 150 200 250 300 350 400
  * is_day      (is_day) bool 2B False True
  * latitude    (latitude) float64 8B 22.75
  * longitude   (longitude) int64 8B -158
  * sieve_size  (sieve_size) float64 40B 0.2 0.5 1.0 2.0 5.0
  * time        (time) datetime64[ns] 5kB 1988-10-31 1988-12-02 ... 2022-09-02
Data variables: (12/25)
    abnd        (time, is_day, latitude, longitude, depth, sieve_size) float64 440kB ...
    bsal        (time, latitude, longitude, depth) float64 44kB ...
    carb        (time, is_day, latitude, longitude, depth, sieve_size) float64 440kB ...
    chl         (time, latitude, longitude, depth) float64 44kB ...
    d12         (time, latitude, longitude, depth) float64 44kB ...
    dwt         (time, is_day, latitude, longitude, depth, sieve_size) float64 440kB ...
    ...          ...
    svol        (time, is_day, latitude, longitude, depth, sieve_size) float64 440kB ...
    temp        (time, latitude, longitude, depth) float64 44kB ...
    theta       (time, latitude, longitude, depth) float64 44kB ...
    vol         (time, is_day, latitude, longitude, depth, sieve_size) float64 440kB ...
    wwt         (time, is_day, latitude, longitude, depth, sieve_size) float64 440kB ...
    xmiss       (time, latitude, longitude, depth) float64 44kB ...

In [4]:
data_papa = xr.open_dataset(
    "../stations/papa/2_processed/papa_zooplankton.zarr", engine="zarr"
)
data_papa.load()

<xarray.Dataset> Size: 297MB
Dimensions:           (time: 623, is_day: 2, latitude: 12, longitude: 31,
                       depth: 8)
Coordinates:
  * depth             (depth) int64 64B 50 100 150 200 250 300 350 400
  * is_day            (is_day) bool 2B False True
  * latitude          (latitude) float64 96B 46.5 47.5 48.5 ... 55.5 56.5 57.5
  * longitude         (longitude) float64 248B -157.5 -156.5 ... -128.5 -127.5
  * time              (time) datetime64[ns] 5kB 1995-09-24 ... 2020-08-31
Data variables:
    benthos           (time, is_day, latitude, longitude, depth) float64 30MB ...
    chaetognatha      (time, is_day, latitude, longitude, depth) float64 30MB ...
    crustacean        (time, is_day, latitude, longitude, depth) float64 30MB ...
    larvacean         (time, is_day, latitude, longitude, depth) float64 30MB ...
    mesh_size         (time, is_day, latitude, longitude, depth) float64 30MB ...
    others            (time, is_day, latitude, longitude, depth) float64 30MB ...
    small_gelatinous  (time, is_day, latitude, longitude, depth) float64 30MB ...
    thaliacea         (time, is_day, latitude, longitude, depth) float64 30MB ...
    total             (time, is_day, latitude, longitude, depth) float64 30MB ...
    volume_filtered   (time, is_day, latitude, longitude, depth) float64 30MB ...

---

## Vizualization

### BAT station


In [5]:
zoo_bats = (
    data_bats["dry_weight_vol_water_ratio"]
    .dropna("time", how="all")
    .mean("depth")
    .isel(latitude=0, longitude=0)
    .sum("sieve_size")
)
bats_units = data_bats["dry_weight_vol_water_ratio"].pint.quantify().pint.units
zoo_bats_d_n = zoo_bats.to_dataframe().pivot_table(
    index=["time"], columns=["is_day"], values=["dry_weight_vol_water_ratio"]
)
zoo_bats_d_n.columns = ["night", "day"]
# transform 0 to nan
zoo_bats_d_n = zoo_bats_d_n.map(lambda x: None if x == 0 else x)
zoo_bats_d_n

,night,day
time,,
1995-05-10,2.770,NaN
1995-05-11,NaN,2.410
1995-06-13,2.210,1.780
1995-06-14,3.370,NaN
1995-07-11,3.145,2.605
...,...,...
2022-09-17,1.780,NaN
2022-09-18,1.875,NaN
2022-11-23,1.465,NaN


In [6]:
day = zoo_bats_d_n["day"].dropna()
night = zoo_bats_d_n["night"].dropna()
go.Figure(
    data=[
        go.Scatter(x=day.index, y=day, mode="lines", name="Day"),
        go.Scatter(x=night.index, y=night, mode="lines", name="Night"),
    ]
).update_layout(
    title="BATS : zooplankton dry weight by cube meter of sea water",
    xaxis_title="Time",
    yaxis_title="Dry Weight (mg/m3)",
    legend_title="Time of Day",
).show()

### HOT station


In [7]:
zoo_hot = (
    data_hot["dwt"]
    .dropna("time", how="all")
    .mean("depth")
    .isel(latitude=0, longitude=0)
    .sum("sieve_size")
)
hot_units = data_hot["dwt"].pint.quantify().pint.units
zoo_hot = zoo_hot.pint.quantify(hot_units).pint.to("mg/m^3").pint.dequantify()
zoo_hot_d_n = zoo_hot.to_dataframe().pivot_table(
    index=["time"], columns=["is_day"], values=["dwt"]
)
zoo_hot_d_n.columns = ["night", "day"]
# # transform 0 to nan
zoo_hot_d_n = zoo_hot_d_n.map(lambda x: None if x == 0 else x)
zoo_hot_d_n

,night,day
time,,
1994-02-17,3.699338,1.051934
1994-02-18,3.925000,0.604762
1994-02-19,3.259854,3.321583
1994-03-09,2.529457,2.003175
1994-03-10,3.760984,1.545122
...,...,...
2022-07-10,10.020219,7.180867
2022-07-30,13.231735,NaN
2022-07-31,9.892350,5.472701


In [8]:
day = zoo_hot_d_n["day"].dropna()
night = zoo_hot_d_n["night"].dropna()
go.Figure(
    data=[
        go.Scatter(x=day.index, y=day, mode="lines", name="Day"),
        go.Scatter(x=night.index, y=night, mode="lines", name="Night"),
    ]
).update_layout(
    title="HOT : zooplankton dry weight by cube meter of sea water",
    xaxis_title="Time",
    yaxis_title="Dry Weight (mg/m3)",
    legend_title="Time of Day",
).show()

---

## Export the products in netCDF format

First, write the function to prepare the forcing and save it as NetCDF file.


In [9]:
def to_netcdf(zooplankton, station_name, latitude, longitude):
    as_xarray = xr.Dataset.from_dataframe(zooplankton)
    as_xarray = as_xarray.expand_dims(
        {"latitude": [latitude], "longitude": [longitude], "layer": [0]}
    )
    as_xarray["time"].attrs = {
        "axis": "T",
        "standard_name": "time",
        "long_name": "time",
    }
    as_xarray["night"].attrs = {
        "units": "mg/m3",
        "description": "Zooplankton dry weight by cube meter of sea water at night",
    }
    as_xarray["day"].attrs = {
        "units": "mg/m3",
        "description": "Zooplankton dry weight by cube meter of sea water during the day",
    }
    as_xarray.attrs = {
        "station": station_name,
        "creation date": datetime.now().isoformat(),
        "layer": "epipelagic",
    }
    as_xarray["layer"].attrs = {
        "flag_values": [0, 1, 2],
        "flag_meanings": "epipelagic, upper-mesopelagic, lower-mesopelagic",
        "desc": "The layer depth according to Seapodym definition",
        "standard_name": "layer",
        "long_name": "Layer depth",
        "axis": "Z",
    }
    as_xarray["latitude"].attrs = {
        "standard_name": "latitude",
        "axis": "Y",
        "units": "degrees_north",
        "long_name": "Latitude",
    }
    as_xarray["longitude"].attrs = {
        "standard_name": "longitude",
        "axis": "X",
        "units": "degrees_east",
        "long_name": "Longitude",
    }
    as_xarray = as_xarray.transpose("time", "latitude", "longitude", "layer")
    as_xarray.to_netcdf(f"{station_name}_zooplankton.nc")
    return as_xarray

Then export all the stations to NetCDF format.


In [10]:
to_netcdf(zoo_bats_d_n, "Bats", latitude=31.6, longitude=-64.2)

<xarray.Dataset> Size: 13kB
Dimensions:    (latitude: 1, longitude: 1, layer: 1, time: 538)
Coordinates:
  * latitude   (latitude) float64 8B 31.6
  * longitude  (longitude) float64 8B -64.2
  * layer      (layer) int64 8B 0
  * time       (time) datetime64[ns] 4kB 1995-05-10 1995-05-11 ... 2022-12-13
Data variables:
    night      (time, latitude, longitude, layer) float64 4kB 2.77 nan ... 2.775
    day        (time, latitude, longitude, layer) float64 4kB nan 2.41 ... 0.905
Attributes:
    station:        Bats
    creation date:  2025-03-21T11:06:28.554425
    layer:          epipelagic

In [11]:
to_netcdf(zoo_hot_d_n, "Hot", latitude=22.75, longitude=-158)

<xarray.Dataset> Size: 13kB
Dimensions:    (latitude: 1, longitude: 1, layer: 1, time: 546)
Coordinates:
  * latitude   (latitude) float64 8B 22.75
  * longitude  (longitude) int64 8B -158
  * layer      (layer) int64 8B 0
  * time       (time) datetime64[ns] 4kB 1994-02-17 1994-02-18 ... 2022-09-02
Data variables:
    night      (time, latitude, longitude, layer) float64 4kB 3.699 ... 6.41
    day        (time, latitude, longitude, layer) float64 4kB 1.052 ... 8.728
Attributes:
    station:        Hot
    creation date:  2025-03-21T11:06:29.381449
    layer:          epipelagic